## Cleaning Regional Dengue Clusters Data

In [1]:
import pandas as pd
import numpy as np
import glob
import geopandas as gpd
import fiona

import datetime

from sklearn.impute import SimpleImputer

fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default

## Read in Planning Area File

Read in as GeoDataFrame

In [2]:
planning_area = gpd.read_file('../datasets/dengue_cases_regional/planning-boundary-area.kml')

In [3]:
planning_area[planning_area['PLN_AREA_N']=='JURONG WEST'].plot()

KeyError: 'PLN_AREA_N'

In [ ]:
# convert polygon shape into point
planning_area['centroid'] = planning_area['geometry'].centroid

## Read in Dengue Clusters Files

Read in each file and append into single file, then convert to GeoDataFrame

In [ ]:
dengue_cluster_filelist = glob.glob('../datasets/dengue_cases_regional/*-clusters.csv')

df = []

for file in dengue_cluster_filelist:

    temp = pd.read_csv(file, names=['num_of_cases', 
                                  'street_address', 
                                  'latitude', 
                                  'longitude', 
                                  'cluster_num', 
                                  'recent_cases_in_cluster', 
                                  'total_cases_in_cluster',
                                  'date',
                                  'month'])
    df.append(temp)
    
df = pd.concat(df, ignore_index=True)

### Data Dictionary

| | |
|----|----|
|Number Of Cases|Number of reported dengue cases at this location |
|Street Address|Street address where dengue cases are reported (down to the apartment block level) |
|Latitude|Latitude of the street address |
|Longitude|Longitude of the street address |
|Cluster Number|Every dengue cluster is labelled with a serial number. However, this serial number cannot be used as an unique identifier because (1) the serial number is reused in other snapshots and (2) the serial number will change throughout the cluster's lifetime. |
|Recent Cases In Cluster**|Number of dengue cases with onset in the last 2 weeks |
|Total Cases In Cluster| Total number of dengue cases reported in this cluster |
|Date|Date string in YYMMDD format |
|Month Number|Index number of the month, where 1=January and 12=December |

In [ ]:
df.info()
# Observation: no null values, need to convert date to datetime dtype

In [ ]:
df.describe()

In [ ]:
# convert date to datetime dtype
df['date'] = pd.to_datetime(df['date'], format='%y%m%d')

In [ ]:
# check which ones are sundays
df['date'].dt.dayofweek.value_counts()

In [ ]:
# check out case distribution by weekdays
df['weekday'] = df['date'].dt.dayofweek
df.groupby('weekday').describe()

Observation:

- Data is not aligned to sunday of the week, need to impute.

In [ ]:
# create geometry column
df['geometry'] = gpd.points_from_xy(x=df['longitude'], y=df['latitude'])

# convert to GeoDataFrame
df = gpd.GeoDataFrame(df)
df.set_crs(epsg=4326, inplace=True)

In [12]:
jdf = gpd.sjoin(df, planning_area, how='left')

In [13]:
jdf.head(50)

,num_of_cases,street_address,latitude,longitude,cluster_num,recent_cases_in_cluster,total_cases_in_cluster,date,month,weekday,...,icon,PLN_AREA_N,PLN_AREA_C,CA_IND,REGION_N,REGION_C,INC_CRC,FMEL_UPD_D,snippet,centroid
0,4,bishan street 22 (block 232),1.358286,103.845226,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
1,7,bishan street 22 (block 233),1.358639,103.845259,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
2,3,bishan street 22 (block 234),1.358390,103.845955,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
3,1,bishan street 22 (block 235),1.358719,103.846477,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
4,4,bishan street 22 (block 236),1.359041,103.846849,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
5,1,bishan street 22 (block 237),1.359335,103.847097,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
6,2,bishan street 22 (block 293),1.356245,103.845879,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
7,2,bishan street 23 (block 220),1.357568,103.848743,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
8,1,bishan street 23 (block 222),1.358069,103.848079,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)
9,5,bishan street 23 (block 223),1.358557,103.848049,1,24,83,2015-07-03,7,4,...,NaN,BISHAN,BS,N,CENTRAL REGION,CR,F06B9400ED89EF69,20191206144714,,POINT (103.84044 1.35496)


In [14]:
jdf.columns

Index(['num_of_cases', 'street_address', 'latitude', 'longitude',
       'cluster_num', 'recent_cases_in_cluster', 'total_cases_in_cluster',
       'date', 'month', 'weekday', 'geometry', 'index_right', 'Name',
       'description', 'timestamp', 'begin', 'end', 'altitudeMode',
       'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon',
       'PLN_AREA_N', 'PLN_AREA_C', 'CA_IND', 'REGION_N', 'REGION_C', 'INC_CRC',
       'FMEL_UPD_D', 'snippet', 'centroid'],
      dtype='object')

In [15]:
# extract only needed columns
jdf = jdf[['cluster_num',
           'recent_cases_in_cluster',
           'date', 'REGION_N', 'centroid']]

## Aggregate Data by Date and Regions

In [16]:
# aggregate at cluster level
jdf = jdf.groupby(['date', 'cluster_num']).agg({'recent_cases_in_cluster': 'mean', 'REGION_N':'first', 'centroid':'first'}).reset_index()

# aggregate to region level
dengue_clusters_by_region = jdf.groupby(['date', 'REGION_N']).sum(numeric_only=True).reset_index()

# pivot data to create one datapoint per date (only keep num_of_cases and recent_cases)
dengue_clusters_by_region = dengue_clusters_by_region.pivot(index='date', columns='REGION_N', values=['recent_cases_in_cluster'])

# create full list of dates and reindex dataframe
dates = pd.date_range(start=dengue_clusters_by_region.index[0], end=dengue_clusters_by_region.index[-1], freq='D')
dengue_clusters_by_region = dengue_clusters_by_region.reindex(dates)

In [17]:
# impute missing data for total_cases
dengue_clusters_by_region['recent_cases_in_cluster'] = dengue_clusters_by_region['recent_cases_in_cluster'].interpolate(method='slinear').round(0)

In [18]:
# extract only the sunday figures
dengue_clusters_by_region = dengue_clusters_by_region[dengue_clusters_by_region.index.weekday == 6]

In [31]:
# save planning area centroids to file
regions = jdf.groupby('REGION_N').agg({'centroid':'first'}).reset_index()

# rename column from centroid to geometry for easier plotting in geopandas later on
regions.columns = ['REGION_N', 'geometry']

regions.to_csv('../datasets/regions.csv', index=False)

In [19]:
# rename index to 'date'
dengue_clusters_by_region.rename_axis('date', inplace=True)

# flatten columns
dengue_clusters_by_region.columns = [f"{i}_{j.split(' ')[0].lower()}" for i, j in dengue_clusters_by_region.columns]

# save to file
dengue_clusters_by_region.to_csv('../datasets/dengue_clusters_by_region.csv', index=True)

## Unused code for Planning Area

In [155]:
dengue_clusters_by_planningarea = jdf.groupby(['date', 'PLN_AREA_N']).sum(numeric_only=True)

In [156]:
dengue_clusters_by_planningarea.drop(columns=['latitude', 'longitude', 'cluster_num'], inplace=True)

In [157]:
dengue_clusters_by_planningarea.to_csv('../datasets/dengue_clusters_by_planningarea.csv', index=False)

In [238]:
def interpolate_recent_cases(df):
    
    for i in df.index:
        #for region in range(5):
            
            #print(row['recent_cases_in_cluster'][region])
            
            # test if recent_cases is alredy filled, if not filled, then run the interpolation
            #if not np.isnan(row['recent_cases_in_cluster'][region]):
        try: # try-except structure to catch cases where the interpolation data is unavailable
            # recent_cases will be interpolated by adding the current latest cases minus off the latest cases from 2 weeks ago
           
            cases_today = df.loc[i, 'num_of_cases']
            recent_cases_yest = df.loc[i-datetime.timedelta(days=1), 'recent_cases_in_cluster']
            cases_2weeksback = df.loc[i-datetime.timedelta(days=14), 'num_of_cases']
            
            df.loc[i, 'recent_cases_in_cluster'] = (recent_cases_yest + cases_today - cases_2weeksback).apply(lambda x: max(x,0)).values

        except: # except clause to return null, so the recent_cases will remain NaN
            pass
                
    return df